In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
df = pd.read_csv(r'C:\Users\joaoh\Desktop\Faculdade\Projetos\Challeginge-F5-ClickBus\desafio clickbus\desafio clickbus\dados_desafio_fiap\hash\df_t.csv')
df.head()

,nk_ota_localizer_id,fk_contact,date_purchase,time_purchase,place_origin_departure,place_destination_departure,place_origin_return,place_destination_return,fk_departure_ota_bus_company,fk_return_ota_bus_company,gmv_success,total_tickets_quantity_success
0,bc02d5245bec63b30ff1102fa273fc03f58bc9cc3f674e...,a7218ff4ee7d37d48d2b4391b955627cb089870b934912...,2018-12-26,15:33:35,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,50e9a8665b62c8d68bccc77c7c92431a1aa26ccbd38ed4...,0,0,8527a891e224136950ff32ca212b45bc93f69fbb801c3b...,1,89.09,1
1,5432f12612dd5d749b3be880e779989cf63b5efa4bcc4e...,37228485e0dc83d84d1bcd1bef3dc632301bf6cb22c8b5...,2018-12-05,15:07:57,10e4e7caf8b078429bb1c80b1a10118ac6f963eff098fd...,e6d41d208672a4e50b86d959f4a6254975e6fb9b088116...,0,0,36ebe205bcdfc499a25e6923f4450fa8d48196ceb4fa0c...,1,155.97,1
2,fb3caed9b2f1b6016d45ccddb19095476e61a2c85faa8e...,3467ec081e2421e72c96e7203b929d21927fd00b6b5f28...,2018-12-21,18:41:54,7688b6ef52555962d008fff894223582c484517cea7da4...,8c1f1046219ddd216a023f792356ddf127fce372a72ec9...,0,0,ec2e990b934dde55cb87300629cedfc21b15cd28bbcf77...,1,121.99,1
3,4dc44a6dd592b702feccb493d192210c86965aee684529...,ab3251a2be0f69713b8f97b0e9d1579e31551f4fd4facf...,2018-12-06,14:01:38,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,d6acb3c1a79e57bcc03d976cb4d98f56edccd4cf426392...,0,0,5f9c4ab08cac7457e9111a30e4664920607ea2c115a143...,1,55.22,1
4,aa34ed7fd0a6b405df2df1bf9f8d68e6df9b9a868a6181...,ceea0de820a6379f2c4215bddaec66c33994b304607e56...,2021-02-23,20:08:25,7688b6ef52555962d008fff894223582c484517cea7da4...,23765fc69c4e3c0b10f5d15471dc2245e2a19af16b513f...,0,0,48449a14a4ff7d79bb7a1b6f3d488eba397c36ef25634c...,1,45.31,1


In [3]:

nomes_portugues = ["ID_compra",
"ID_cliente",
"Data_compra",
"Hora_compra",
"Origem_da_viagem_de_ida",
"Destino_da_viagem_de_ida",
"Origem_da_viagem_de_volta",
"Destino_da_viagem_de_volta",
"Viação_da_viagem_de_ida",
"Viação_da_viagem_de_volta",
"Valor_total_da_compra",
"Quantidade_de_passagens"]

df.columns = nomes_portugues



In [5]:
# Alvo
y = df["Valor_total_da_compra"]

# Features (excluindo IDs e colunas que não ajudam na predição)
X = df.drop(columns=[
    "ID_compra", "ID_cliente", "Valor_total_da_compra"
])


In [6]:
# Converter colunas de data e hora
X["Data_compra"] = pd.to_datetime(X["Data_compra"])
X["Ano"] = X["Data_compra"].dt.year
X["Mes"] = X["Data_compra"].dt.month
X["Dia"] = X["Data_compra"].dt.day
X["Hora"] = pd.to_datetime(X["Hora_compra"]).dt.hour

# Drop colunas originais de data/hora
X = X.drop(columns=["Data_compra", "Hora_compra"])

# Identificar tipos
cat_features = ["Origem_da_viagem_de_ida", "Destino_da_viagem_de_ida", 
                "Origem_da_viagem_de_volta", "Destino_da_viagem_de_volta", 
                "Viação_da_viagem_de_ida", "Viação_da_viagem_de_volta"]

num_features = ["Quantidade_de_passagens", "Ano", "Mes", "Dia", "Hora"]


C:\Users\joaoh\AppData\Local\Temp\ipykernel_14580\2611565147.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  X["Hora"] = pd.to_datetime(X["Hora_compra"]).dt.hour


In [11]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features)
    ]
)


In [12]:
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(n_estimators=100, random_state=42))
])


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)


In [ ]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")


In [ ]:
nova_viagem = pd.DataFrame([{
    "Origem_da_viagem_de_ida": "São Paulo",
    "Destino_da_viagem_de_ida": "Rio de Janeiro",
    "Origem_da_viagem_de_volta": "Rio de Janeiro",
    "Destino_da_viagem_de_volta": "São Paulo",
    "Viação_da_viagem_de_ida": "123_onibus",
    "Viação_da_viagem_de_volta": "123_onibus",
    "Quantidade_de_passagens": 2,
    "Ano": 2025, "Mes": 8, "Dia": 19, "Hora": 14
}])

preco_previsto = model.predict(nova_viagem)
print("Valor previsto da passagem:", preco_previsto[0])
